# Geoprocessing

Spatial data comes in a rich variety of forms and corresponding file formats. At the beginning of most geocomputational workflows, one is typically reading these different formats and applying different forms of spatial data processing (or geoprocessing) methods to the data.

In this notebook we cover a subset of geoprocessing methods:

- Reading geopackages and shapefiles
- Coordinate reference system transformations
- Spatial joins
- Spatial aggregations

Along the way we introduce the package [geopandas](https://geopandas.org) which provides key spatial data processing functionality.

~~## Reading GeoPackages~~

~~We begin with our first dataset that contains socieconomic data for census tracts in Southern California. The data are for the census year 2010 and are provided from the package [GeoSNAP](https://github.com/spatialucr/geosnap) as a **GeoPackage**.~~

## GeoParquet

**dont use legacy file formats :)**

Geoparquet is a new(ish)  file format based on apache arrow. It is extremely performant for spatial data in both read/write speeds and on-disk storage

We will use **geopandas** to read the GeoPackage for our data:


In [ ]:
import geopandas as gpd

In [ ]:
socal = gpd.read_parquet("data/scag_region.parquet") 

In [ ]:
type(socal)

In [ ]:
socal.columns

In [ ]:
socal.head()

In [ ]:
socal.shape 

This tells us there are 4580 census tracts and 194 attributes measured for each tract.

In [ ]:
socal.columns

At least one of the attributes (columns) in the GeoDataFrame holds the geometries for the observations. In our case, this column is named `geometry`:

In [ ]:
socal.geometry

This shows us that each of our observations is encoded as a Polygon geometry. 

We can call the `plot` method of the GeoDataFrame to visualize these geometries:

In [ ]:
socal.plot('n_total_pop')

Note that the tract identifier column is `geoid` which encodes the [11-digit FIPS code](https://www.fcc.gov/general/form-477-census-tract-information).

~~## Reading shapefiles~~

~~[shapefiles](https://doc.arcgis.com/en/arcgis-online/reference/shapefiles.htm) are a common legacy spatial data format for vector data. 
Let's explore a second data set that encodes mental health provider locations in Riverside County, California as points in a shapefile:~~

## Definitely **do not** use shapefiles

unless you are forced to. Do not ever provide your data to someone else in shapefile format :). You can use geopandas to read/write legacy formats but we wont bother covering them here. Avoid whenever possible

In [ ]:
socal.to_parquet('socal.parquet')

### Reading a point ~~shapefile~~

In [ ]:
clinics_df = gpd.read_parquet('data/behavioralHealth.parquet')
clinics_df.columns

In [ ]:
clinics_df.head()

In [ ]:
clinics_df.plot()

In [ ]:
clinics_df.geometry

In [ ]:
clinics_df.ADDRESS

In [ ]:
len(clinics_df.ADDRESS.unique()) # note some clinics have the same address (different types of services provided at same location)

## Integrating spatial datasets
Let's assume the role of a public health epidemiologist who is interested in equity issues surrounding access to mental health services. We have two datasets to work with here:

- the clinics (points in Riverside County)
- the tracts for all of Southern California

We will thus want to narrow down the focus to Riverside county. So first, we need to extract the tracts for the county from the larger region:


## Extracting Riverside County Tracts

In [ ]:
socal.plot()

In [ ]:
rc = socal[socal['geoid'].str.match("^06065")]

rc.plot()

## Spatial Joins

With the two datasets in hand, we would like to know the answer to the following question:

> How many clinics are in each census tract in Riverside County?

To get at this answer will can use a [spatial join](https://gisgeography.com/spatial-join/). 
The idea here is to create a linkage between each clinic (point) and the tract (polygon) that the clinic lies within.

The answer to our question will be the number of matches found for each census tract.

In geopandas the spatial join is done with the `sjoin` method of the GeoDataFrame:


In [ ]:
clinics_tracts = gpd.sjoin(clinics_df, rc, op='within')

We see a warning here, that alerts us to a mismatch between the CRS: [Coordinate Reference Systems](https://www.w3.org/2015/spatial/wiki/Coordinate_Reference_Systems) of the two GeoDataFrames. We can inspect these to see what is going on:

In [ ]:
clinics_df.crs

In [ ]:
rc.crs

So the clinics have a unit of feet, while the tracts are in degrees. The spatial join will be incorrect because of this since the observations from the two dataframes are in different coordinate systems:

In [ ]:
clinics_tracts

To rectify this, we need to have both GeoDataFrames in the same CRS. Let's change the CRS of the tracts GeoDataFrame which is currently:


In [ ]:
rc.plot()

To change this, we notice the helpful suggestion in the warning above to use the `to_crs` method. Let's try it to see how this works:

In [ ]:
rc.to_crs(clinics_df.crs).plot()

Comparison of the two plots shows us that the CRS has been changed. We did not assign the object yet (which is good practice when experimenting). But it does what we need, so lets assign the result to redefine our tracts GeoDataFrame:

In [ ]:
rc = rc.to_crs(clinics_df.crs)

In [ ]:
rc.crs

In [ ]:
clinics_tracts = gpd.sjoin(clinics_df, rc, op='within')

In [ ]:
clinics_tracts

Now we see the spatial join has worked. There are 28 matches (one for each clinic) and this is stored in a new GeoDataFrame called: `clinics_tracts`.

## Determine the number of clinics in each tract

Returning to our question, we can find the number of clinics in each of the tracts in this new GeoDataFrame by using the `group_by` method:

In [ ]:
clinics_tracts[['geoid', 'index_right']].groupby('geoid').agg('count')

This works, so let's create another DataFrame to store these counts:

In [ ]:
ct = clinics_tracts[['geoid', 'index_right']].groupby('geoid').agg('count')
ct.shape

## Table Join



We now know for the tracts that contain at least one clinic, how many clinics they contain. This implies that the tracts that are not in the `ct` GeoDataFrame do not contain a clinic. What we would like to have is an additional attribute on our `rc` tracts GeoDataFrame that stores the number of clinics in each tract - for all tracts in Riverside County.

We can do this in two remaining steps:

- a table join on the `geoid` column 
- setting NA values to 0

In [ ]:
rc.merge(ct, on='geoid')

Note that this isn't quite what we want as it only matches the 16 tracts containing clinics. We can change the `how` argument to `outer` to include all tracts:

In [ ]:
rc.merge(ct, on='geoid', how='outer')

Ok now we have all the tracts covered. But if you scroll to the right we will see `NaN` values for the tracts that do not contain a clinic. What remains is to replace the 

In [ ]:
rc.merge(ct, on='geoid', how='outer').fillna(0)

Great. Now we save our work and rename the column holding the clinic count:

In [ ]:
rc = rc.merge(ct, on='geoid', how='outer').fillna(0)

In [ ]:
rc.plot('clinics', scheme='quantiles')

In [ ]:
rc['index_right'].sum()

In [ ]:
rc.rename(columns={'index_right': 'clinics'}, inplace=True)

In [ ]:
rc.clinics.sum()

In [ ]:
socal_wgs = socal.to_crs(4326)

In [ ]:
socal_wgs.estimate_utm_crs()

In [ ]:
type(rc)

<div class="alert alert-success" style="font-size:120%">
<b>Exercise</b>: <br>
Create a shapefile for each of the counties in the Southern California dataframe, and write to <code>data/FIPSCODE.shp</code>. Where <code>FIPSCODE</code> is the 5-digit
<a href="https://www.nrcs.usda.gov/wps/portal/nrcs/detail/ca/home/?cid=nrcs143_013697">FIPS Code for the county</a>.
</div>

In [ ]:
# %load solutions/00.py

---

<a rel="license" href="http://creativecommons.org/licenses/by-nc-
sa/4.0/"><img alt="Creative Commons License" style="border-width:0"
src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span
xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Geoprocessing</span> by <a xmlns:cc="http://creativecommons.org/ns#"
href="http://sergerey.org" property="cc:attributionName"
rel="cc:attributionURL">Serge Rey</a> is licensed under a <a
rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative
Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.